## Load Packages & Set Paths

In [17]:
import pandas as pd
import numpy as np
from pathlib import Path
import duckdb

REPO_ROOT = Path().resolve().parents[0]
DB_PATH = REPO_ROOT / "dbt_project" / "dev.duckdb"
print(REPO_ROOT)
print(DB_PATH)

/Users/samharrison/Documents/data_sci/gk_performance_tracker/gk_performance_tracker
/Users/samharrison/Documents/data_sci/gk_performance_tracker/gk_performance_tracker/dbt_project/dev.duckdb


## Connect to `duckdb` DB & Create Query-Function

In [20]:
# Connect to duckdb
con = duckdb.connect(str(DB_PATH), read_only=False)
con.execute("PRAGMA enable_progress_bar=true;")

# Create query-function
def q(sql: str, *params):
    """Run a SQL query and return a pandas DataFrame."""
    return con.execute(sql, params).df()

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Example usage of query-function
q("""
select *
from stg_matchlogs__parsed
where minutes_played > 0
and competition = 'Premier League' 
""")

,source_table,goalkeeper,season,match_date,competition,round,venue,result,team,opponent,game_started,minutes_played,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_clean_sheets,gk_psxg,gk_pens_att,gk_pens_allowed,gk_pens_saved,gk_pens_missed,gk_passes_launched,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_passes_length_avg,gk_goal_kicks,gk_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_def_actions_outside_pen_area,gk_avg_distance_def_actions
0,alisson_2025_2026,alisson,2025_2026,2025-08-15,Premier League,Matchweek 1,Home,W 4–2,Liverpool,Bournemouth,Y,90,3.0,2.0,1.0,0.0,0.8,0.0,0.0,0.0,0.0,11.0,6.0,41.0,4.0,31.7,1.0,0.00,5.0,16.0,1.0,0.0,7.0
1,alisson_2025_2026,alisson,2025_2026,2025-08-25,Premier League,Matchweek 2,Away,W 3–2,Liverpool,Newcastle Utd,Y,90,3.0,2.0,1.0,0.0,1.9,0.0,0.0,0.0,0.0,16.0,6.0,36.0,4.0,35.1,2.0,20.00,66.0,19.0,1.0,5.0,16.4
2,alisson_2025_2026,alisson,2025_2026,2025-08-31,Premier League,Matchweek 3,Home,W 1–0,Liverpool,Arsenal,Y,90,1.0,0.0,1.0,1.0,0.6,0.0,0.0,0.0,0.0,13.0,4.0,48.0,3.0,31.8,2.0,10.00,39.5,12.0,0.0,2.0,18.0
3,alisson_2025_2026,alisson,2025_2026,2025-09-14,Premier League,Matchweek 4,Away,W 1–0,Liverpool,Burnley,Y,90,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,22.0,3.0,25.3,3.0,0.00,15.7,3.0,0.0,0.0,16.0
4,alisson_2025_2026,alisson,2025_2026,2025-09-20,Premier League,Matchweek 5,Home,W 2–1,Liverpool,Everton,Y,90,2.0,1.0,1.0,0.0,0.6,0.0,0.0,0.0,0.0,11.0,4.0,45.0,5.0,27.5,2.0,10.00,41.0,18.0,0.0,0.0,12.3
5,alisson_2025_2026,alisson,2025_2026,2025-09-27,Premier League,Matchweek 6,Away,L 1–2,Liverpool,Crystal Palace,Y,90,7.0,2.0,5.0,0.0,2.6,0.0,0.0,0.0,0.0,5.0,0.0,47.0,8.0,25.0,5.0,10.00,28.6,8.0,1.0,1.0,13.3
6,alisson_2025_2026,alisson,2025_2026,2025-11-22,Premier League,Matchweek 12,Home,L 0–3,Liverpool,Nott'ham Forest,Y,90,7.0,3.0,4.0,0.0,2.5,0.0,0.0,0.0,0.0,1.0,1.0,26.0,9.0,21.9,2.0,0.00,14.0,12.0,2.0,0.0,10.3
7,alisson_2025_2026,alisson,2025_2026,2025-11-30,Premier League,Matchweek 13,Away,W 2–0,Liverpool,West Ham,Y,90,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,24.0,1.0,24.4,3.0,0.00,10.0,18.0,1.0,3.0,17.3
8,alisson_2025_2026,alisson,2025_2026,2025-12-03,Premier League,Matchweek 14,Home,D 1–1,Liverpool,Sunderland,Y,90,5.0,1.0,4.0,0.0,0.4,0.0,0.0,0.0,0.0,5.0,1.0,30.0,5.0,26.5,1.0,0.00,21.0,10.0,0.0,2.0,16.3
9,alisson_2025_2026,alisson,2025_2026,2025-12-06,Premier League,Matchweek 15,Away,D 3–3,Liverpool,Leeds United,Y,90,5.0,3.0,2.0,0.0,2.1,1.0,1.0,0.0,0.0,13.0,2.0,34.0,2.0,33.2,0.0,NaN,NaN,11.0,0.0,0.0,13.7


## Template Dashboard Views

In [37]:
# Example usage of query-function
q("""
select
    goalkeeper,
    team,
    
    -- Performance
    count(*) as matches_played,    
    sum(gk_clean_sheets) as clean_sheets,   
    sum(gk_goals_against) as ga,  
    
    -- Shot-stopping  
    sum(gk_saves) as saves,  
    sum(gk_shots_on_target_against) as shots_on_target_against,
    round(sum(gk_saves) / sum(gk_shots_on_target_against), 3) * 100 as save_pct,
    sum(gk_psxg) - sum(gk_goals_against) as pxsg_minus_ga,
    
    -- Crossing
    round(sum(gk_crosses) / sum(minutes_played), 3) * 100 as crosses_faced_p90,
    round(sum(gk_crosses_stopped) / sum(gk_crosses), 3) * 100 as crosses_stopped_pct,  

    -- Passing
    round(sum(gk_passes) / sum(minutes_played), 3) * 100 as pass_att_p90,
    round(sum(gk_passes_completed_launched) / sum(gk_passes_launched), 3) * 100 as long_kick_pass_completion_pct,  

    -- Sweeper
    round(sum(gk_def_actions_outside_pen_area) / sum(minutes_played), 3) * 100 as def_actions_outside_pen_area,
    sum(gk_def_actions_outside_pen_area * gk_avg_distance_def_actions)
      / nullif(sum(gk_def_actions_outside_pen_area), 0)
        as avg_distance_def_actions
  
from stg_matchlogs__parsed
where minutes_played > 0
and competition = 'Premier League' 

group by
    goalkeeper,
    team
  """)


,goalkeeper,team,matches_played,clean_sheets,ga,saves,shots_on_target_against,save_pct,pxsg_minus_ga,crosses_faced_p90,crosses_stopped_pct,pass_att_p90,long_kick_pass_completion_pct,def_actions_outside_pen_area,avg_distance_def_actions
0,alphonse_areola,West Ham,11,0.0,18.0,40.0,58.0,69.0,-0.6,21.2,4.3,25.3,30.5,0.1,31.000000
1,alisson,Liverpool,11,4.0,14.0,20.0,34.0,58.8,-1.9,14.2,5.0,39.4,34.0,1.4,16.328571
